<a href="https://colab.research.google.com/github/aayush-shah14/Letter_Recognition/blob/main/Letter_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


In [ ]:
import io
data_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
data_val = pd.read_csv(io.BytesIO(uploaded['val.csv']))
data_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:
data_test

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,3,5,4,4,2,7,8,5,4,7,7,7,6,9,2,5
1,4,6,6,6,4,7,5,4,4,8,3,7,3,7,4,8
2,3,7,4,5,3,8,7,7,6,7,6,7,3,8,4,7
3,3,6,4,4,2,5,12,7,3,12,6,3,1,10,4,7
4,3,7,4,5,3,7,6,6,6,6,5,9,2,9,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,2,3,4,1,2,6,8,1,6,10,6,9,3,8,2,8
3496,6,9,8,7,5,9,8,1,8,14,5,6,5,8,5,9
3497,4,11,6,8,6,6,10,4,5,10,9,4,4,10,4,7
3498,1,0,2,1,0,7,7,11,0,5,6,8,4,8,0,8


In [ ]:
x_train = np.array(data_train.loc[:, data_train.columns != 'letter'])
x_val = np.array(data_val.loc[:, data_val.columns != 'letter'])
#_train = data_train.loc[:, data_train.columns != 'letter']
data_train['train'] = 1
data_val['train'] = 0
combined = pd.concat([data_train, data_val])
df_letter = pd.get_dummies(combined['letter'], prefix='letter', drop_first=False)
combined = pd.concat([combined,df_letter], axis = 1)
combined = combined.drop('letter', axis=1)
y_train = combined[combined["train"] ==1]
y_val = combined[combined["train"] ==0]
y_train.drop(["train"], axis = 1, inplace=True)
y_val.drop(["train"], axis = 1, inplace=True)
y_train = np.array(y_train.iloc[:,16:])
y_val = np.array(y_val.iloc[:,16:])


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(16,), name ='layer1'),
        layers.Dense(64, activation="relu", name="layer2"),
        layers.Dense(26, activation='softmax', name="layer3"),
    ]
)

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer2 (Dense)               (None, 64)                1088      
_________________________________________________________________
layer3 (Dense)               (None, 64)                4160      
_________________________________________________________________
layer4 (Dense)               (None, 26)                1690      
Total params: 6,938
Trainable params: 6,938
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    x = x_train,
    y = y_train,
    batch_size = 100,
    epochs = 200,
    verbose = 1,
    validation_data = (x_val, y_val),
    shuffle = True, # doesn't matter if we have only one epoch or no batches,
)

Epoch 1/200
130/130 [==============================] - 1s 2ms/step - loss: 3.7416 - accuracy: 0.1100 - val_loss: 1.9466 - val_accuracy: 0.5117
Epoch 2/200
130/130 [==============================] - 0s 2ms/step - loss: 1.6988 - accuracy: 0.5678 - val_loss: 1.2536 - val_accuracy: 0.6617
Epoch 3/200
130/130 [==============================] - 0s 1ms/step - loss: 1.1753 - accuracy: 0.6794 - val_loss: 1.0459 - val_accuracy: 0.7114
Epoch 4/200
130/130 [==============================] - 0s 1ms/step - loss: 1.0009 - accuracy: 0.7276 - val_loss: 0.9460 - val_accuracy: 0.7357
Epoch 5/200
130/130 [==============================] - 0s 2ms/step - loss: 0.8925 - accuracy: 0.7507 - val_loss: 0.8394 - val_accuracy: 0.7617
Epoch 6/200
130/130 [==============================] - 0s 2ms/step - loss: 0.8137 - accuracy: 0.7704 - val_loss: 0.7827 - val_accuracy: 0.7766
Epoch 7/200
130/130 [==============================] - 0s 1ms/step - loss: 0.7446 - accuracy: 0.7911 - val_loss: 0.7698 - val_accuracy: 0.7734

In [ ]:
results = model.predict(x_val,batch_size=1)

In [ ]:
x_test = data_test.to_numpy()

In [ ]:
x_test.shape

(3500, 16)

In [ ]:
results = model.predict(x_test,batch_size = 1)
results


array([[6.48909149e-13, 1.26173285e-25, 1.89953618e-15, ...,
        6.92847399e-29, 2.76252840e-11, 8.35231642e-32],
       [5.22297512e-08, 2.87269115e-11, 1.98534565e-07, ...,
        3.23657856e-09, 2.97561553e-19, 5.67180045e-08],
       [6.73516197e-17, 1.82792337e-10, 4.11968394e-05, ...,
        2.34858660e-10, 1.37200825e-13, 4.89851859e-17],
       ...,
       [1.03263127e-19, 9.84000928e-16, 1.18479923e-18, ...,
        3.49816503e-24, 2.21275347e-14, 1.26459178e-29],
       [8.51673311e-16, 7.22091266e-33, 2.19938211e-26, ...,
        1.11195135e-33, 9.87614807e-17, 0.00000000e+00],
       [7.14060802e-37, 0.00000000e+00, 1.97932045e-17, ...,
        8.65843006e-26, 1.66916963e-13, 0.00000000e+00]], dtype=float32)

In [ ]:
b = []
for i in range(len(x_test)):
  t = np.argmax(results[i])
  b.append(chr(65 + t))
d = pd.DataFrame(b)
d.to_csv('result.csv')         
  